In [2]:
# Importing necessary libs
import os
import sys
import numpy as np
import pandas as pd
import cv2
import xmltodict
import json
from tqdm.notebook import tqdm

from pycocotools.coco import COCO

In [3]:
# Setting directories
img_dir = "JPEGImages/"
anno_dir = "Annotations/"
files = os.listdir(anno_dir)


In [ ]:
# Converting pascal voc data format to pandas dataframe
combined = [];
for i in tqdm(range(len(files))):
    annoFile = anno_dir + "/" + files[i];
    f = open(annoFile, 'r');
    my_xml = f.read();
    try:
        anno = dict(dict(xmltodict.parse(my_xml))["annotation"])
        fname = anno["filename"];
        label_str = "";
        if(type(anno["object"]) == list):
            for j in range(len(anno["object"])):
                obj = dict(anno["object"][j]);
                label = anno["object"][j]["name"];
                bbox = dict(anno["object"][j]["bndbox"])
                x1 = bbox["xmin"];
                y1 = bbox["ymin"];
                x2 = bbox["xmax"];
                y2 = bbox["ymax"];
                if(j == len(anno["object"])-1):
                    label_str += x1 + " " + y1 + " " + x2 + " " + y2 + " " + label;
                else:        
                    label_str += x1 + " " + y1 + " " + x2 + " " + y2 + " " + label + " ";
        else:
            obj = dict(anno["object"]);
            label = anno["object"]["name"];
            bbox = dict(anno["object"]["bndbox"])
            x1 = bbox["xmin"];
            y1 = bbox["ymin"];
            x2 = bbox["xmax"];
            y2 = bbox["ymax"];

            label_str += x1 + " " + y1 + " " + x2 + " " + y2 + " " + label
        
        combined.append([fname, label_str])
    except:
        pass
    


In [ ]:
df = pd.DataFrame(combined, columns = ['ID', 'Label'])
df.to_csv("val_labels.csv", index=False)

In [ ]:
# Converting pandas df to COCO format
anno_file = "val_labels.csv"
annotations_path = r"C:\Users\Acer\Downloads\openlogo\annotations"
if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = img_dir
input_annotations_path = anno_file
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path

In [ ]:
tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + 'valid' + ".json";
output_classes_file = output_annotation_folder + "/classes.txt"
if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder)
df = pd.read_csv(input_annotations_path)
columns = df.columns


In [ ]:
list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(' ');
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();

In [ ]:
coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    if i%50==0:
        img_name = df[columns[0]][i];
        labels = df[columns[1]][i];
        tmp = labels.split(' ');
        image_in_path = input_images_folder + "/" + img_name
        print(image_in_path)
        img = cv2.imread(image_in_path, 1)
        h, w, c = img.shape;

        images_tmp = {};
        images_tmp["file_name"] = img_name;
        images_tmp["height"] = h;
        images_tmp["width"] = w;
        images_tmp["id"] = image_id;
        coco_data["images"].append(images_tmp);


        for j in range(len(tmp)//5):
            x1 = int(tmp[j*5+0]);
            y1 = int(tmp[j*5+1]);
            x2 = int(tmp[j*5+2]);
            y2 = int(tmp[j*5+3]);
            label = tmp[j*5+4];
            annotations_tmp = {};
            annotations_tmp["id"] = annotation_id;
            annotation_id += 1;
            annotations_tmp["image_id"] = image_id;
            annotations_tmp["segmentation"] = [];
            annotations_tmp["ignore"] = 0;
            annotations_tmp["area"] = (x2-x1)*(y2-y1);
            annotations_tmp["iscrowd"] = 0;
            annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
            annotations_tmp["category_id"] = anno.index(label);

            coco_data["annotations"].append(annotations_tmp)
        image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4)
outfile.write(json_str)
outfile.close()